In [26]:
# !pip install -U -q kaggle
# !mkdir -p ~/.kaggle
# !echo '{"username":"harrierdb","key":"d35f4ee749dde0ec819807713df28790"}' > ~/.kaggle/kaggle.json
# !chmod 600 ~/.kaggle/kaggle.json
 
# !kaggle competitions download -c tabular-playground-series-oct-2021 --force

 97% 425M/438M [00:02<00:00, 148MB/s]
100% 438M/438M [00:03<00:00, 153MB/s]
  0% 0.00/1.12M [00:00<?, ?B/s]
100% 1.12M/1.12M [00:00<00:00, 71.0MB/s]
 99% 868M/877M [00:06<00:00, 143MB/s]
100% 877M/877M [00:06<00:00, 147MB/s]


In [27]:
import pandas as pd
import numpy as np
!pip install catboost
import gc

In [28]:
data = pd.read_csv('./train.csv.zip', index_col = 0, nrows = 5000)
data_test = pd.read_csv('./test.csv.zip', index_col = 0, nrows = 1000)


In [29]:

data_test['target'] = -1
data.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,...,f246,f247,f248,f249,f250,f251,f252,f253,f254,f255,f256,f257,f258,f259,f260,f261,f262,f263,f264,f265,f266,f267,f268,f269,f270,f271,f272,f273,f274,f275,f276,f277,f278,f279,f280,f281,f282,f283,f284,target
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.205979,0.410993,0.176775,0.223581,0.423543,0.476140,0.413590,0.612021,0.534873,0.147295,0.026177,0.106613,0.200924,0.713191,0.155750,0.557335,0.341702,0.285720,0.230396,0.203957,0.509588,0.706972,1,0.007793,0.247765,0.263750,0.259555,0.231730,0.138379,0.197824,0.054392,0.194153,0.281500,0.034818,0.025334,0.114432,0.139203,0.246157,0.251371,0.701423,...,1,0,0,0,0,0,0,1,0,1,1,0,0,0,1,0,0,1,1,0,0,0,1,1,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,1
1,0.181004,0.473119,0.011734,0.213657,0.619678,0.441593,0.230407,0.686013,0.281971,0.238509,0.493411,0.107277,0.231828,0.457150,0.395200,0.617088,0.459358,0.209225,0.201098,0.199383,0.366578,0.585788,1,0.285311,0.400367,0.162493,0.249365,0.141160,0.133688,0.247906,0.139251,0.216444,0.109674,0.033018,0.017458,0.189336,0.168785,0.184251,0.202753,0.218451,...,1,0,0,1,0,1,0,1,1,0,0,0,1,0,0,0,0,1,1,0,1,1,0,0,0,1,0,0,1,1,0,1,0,0,0,0,0,0,0,1
2,0.182583,0.307431,0.325950,0.207116,0.605699,0.309695,0.493337,0.751107,0.536272,0.286813,0.139532,0.107222,0.247791,0.631949,0.347463,0.642173,0.257763,0.162548,0.327377,0.193583,0.495440,0.636742,0,0.007133,0.309747,0.221081,0.284810,0.230828,0.138271,0.199742,0.060408,0.146746,0.208131,0.035977,0.022631,0.113542,0.274871,0.182770,0.151502,0.570035,...,1,1,0,0,1,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1
3,0.180240,0.494592,0.008367,0.223580,0.760618,0.439211,0.432055,0.776147,0.483958,0.260886,0.147122,0.105433,0.287755,0.455777,0.247971,0.616628,0.335907,0.337025,0.239127,0.176163,0.538269,0.706468,1,0.008829,0.353799,0.219977,0.266858,0.145617,0.138590,0.234925,0.059817,0.140886,0.205023,0.319312,0.009972,0.112291,0.288915,0.332632,0.140831,0.473845,...,1,1,0,1,1,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1
4,0.177172,0.495513,0.014263,0.548819,0.625396,0.562493,0.117158,0.561255,0.077115,0.158321,0.260239,0.102561,0.265285,0.503776,0.269776,0.545945,0.319548,0.278538,0.214922,0.200239,0.534551,0.728652,0,0.004840,0.323546,0.166292,0.285516,0.208651,0.200394,0.198155,0.414729,0.251997,0.193405,0.034490,0.016465,0.197312,0.207429,0.255857,0.139875,0.321039,...,0,0,0,1,1,1,0,1,0,1,0,0,1,1,0,1,0,0,0,1,0,0,0,0,1,1,0,0,1,0,0,1,1,0,1,0,0,1,0,1


In [30]:
df = data.append(data_test)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6000 entries, 0 to 1000999
Columns: 286 entries, f0 to target
dtypes: float64(240), int64(46)
memory usage: 13.1 MB


In [31]:
from sklearn.model_selection import KFold
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score



In [37]:
lgb_params = {
        'objective': 'binary',
        'n_estimators': 20000,
        'random_state': 42,
        'learning_rate': 8e-3,
        'subsample': 0.6,
        'subsample_freq': 1,
        'colsample_bytree': 0.4,
        'reg_alpha': 10.0,
        'reg_lambda': 1e-1,
        'min_child_weight': 256,
        'min_child_samples': 500,
}
xgb_params = {'n_estimators': 10000,
        'learning_rate': 0.03689407512484644,
        'max_depth': 8,
        'colsample_bytree': 0.3723914688159835,
        'subsample': 0.780714581166012,
        'eval_metric': 'auc',
        'use_label_encoder': False,
        'gamma': 0,
        'reg_lambda': 50.0,
        'random_state': 42}

cat_params = {'iterations': 17298,
        'learning_rate': 0.03429054860458741,
        'reg_lambda': 0.3242286463210283,
        'subsample': 0.9433911589913944,
        'random_strength': 22.4849972385133,
        'depth': 8,
        'min_data_in_leaf': 4,
        'leaf_estimation_iterations': 8,
        'task_type':"GPU",
        'bootstrap_type':'Poisson',
        'verbose' : 500,
        'early_stopping_rounds' : 200,
        'eval_metric' : 'AUC'}
lgb = LGBMClassifier(**lgb_params)
xgb = XGBClassifier(**xgb_params)
cat = CatBoostClassifier(**cat_params)

In [39]:


def get_oof(feats, target, test, kfold, clf):
  oof_preds = np.zeros(feats.shape[0])
  sub_preds = np.zeros(test.shape[0])
  for i, (train_idx, valid_idx) in enumerate(kfold.split(feats)):
    train_X, train_y = feats.loc[train_idx], target.loc[train_idx]
    valid_X, valid_y = feats.loc[valid_idx], target.loc[valid_idx]

    clf.fit(train_X, train_y, eval_set = [(valid_X, valid_y)], verbose = 500, early_stopping_rounds = 500, )
    oof_preds[valid_idx] = clf.predict_proba(valid_X)[:,1]
    sub_preds += clf.predict_proba(test)[:,1]
    del train_X, train_y, valid_X, valid_y
    gc.collect()

  evalution_result = roc_auc_score(target, oof_preds)
  print('*'*10)
  print('roc auc score:', evalution_result)
  print('*'*20)
  sub_preds_result = sub_preds / kfold.n_splits
  return oof_preds ,sub_preds_result


In [34]:
feats = df.loc[df['target']!=-1, :].drop('target', axis = 1)
target = df.loc[df['target']!=-1,'target']
test = df.loc[df['target']==-1,:].drop('target', axis = 1)
kfold = KFold(n_splits = 5, shuffle = True, random_state = 2021)


In [35]:
# xgboost, lightgbm, catboost

oof_preds_1, sub_preds_1 = get_oof(feats, target, test, kfold, lgb)
oof_preds_2, sub_preds_2 = get_oof(feats, target, test, kfold, xgb)


Training until validation scores don't improve for 500 rounds.
[500]	training's binary_logloss: 0.572715	valid_1's binary_logloss: 0.564124
[1000]	training's binary_logloss: 0.563261	valid_1's binary_logloss: 0.558674
[1500]	training's binary_logloss: 0.558226	valid_1's binary_logloss: 0.556823
[2000]	training's binary_logloss: 0.554538	valid_1's binary_logloss: 0.556036
[2500]	training's binary_logloss: 0.551533	valid_1's binary_logloss: 0.555614
[3000]	training's binary_logloss: 0.548976	valid_1's binary_logloss: 0.555475
[3500]	training's binary_logloss: 0.546752	valid_1's binary_logloss: 0.555404
[4000]	training's binary_logloss: 0.544795	valid_1's binary_logloss: 0.555387
Early stopping, best iteration is:
[3760]	training's binary_logloss: 0.545699	valid_1's binary_logloss: 0.555256
Training until validation scores don't improve for 500 rounds.
[500]	training's binary_logloss: 0.574733	valid_1's binary_logloss: 0.561744
[1000]	training's binary_logloss: 0.566851	valid_1's binary_l

In [40]:
oof_preds_3, sub_preds_3 = get_oof(feats, target, test, kfold, cat)  #调整eval_set = [(valid_X, valid_y)]

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

0:	learn: 0.8041800	test: 0.8122180	best: 0.8122180 (0)	total: 94.2ms	remaining: 27m 8s
500:	learn: 0.9999830	test: 0.8433140	best: 0.8476583 (353)	total: 57.4s	remaining: 32m 5s
bestTest = 0.8476582766
bestIteration = 353
Shrink model to first 354 iterations.
0:	learn: 0.7652475	test: 0.7740689	best: 0.7740689 (0)	total: 124ms	remaining: 35m 37s
500:	learn: 0.9998797	test: 0.8480409	best: 0.8508911 (123)	total: 58.1s	remaining: 32m 28s
bestTest = 0.8508911133
bestIteration = 123
Shrink model to first 124 iterations.
0:	learn: 0.7667292	test: 0.7708457	best: 0.7708457 (0)	total: 64.7ms	remaining: 18m 38s
500:	learn: 0.9999868	test: 0.8283938	best: 0.8318899 (395)	total: 58.2s	remaining: 32m 32s
bestTest = 0.8318899274
bestIteration = 395
Shrink model to first 396 iterations.
0:	learn: 0.7717876	test: 0.7323637	best: 0.7323637 (0)	total: 66.4ms	remaining: 19m 9s
500:	learn: 0.9999688	test: 0.8186004	best: 0.8200089 (373)	total: 58.1s	remaining: 32m 28s
1000:	learn: 1.0000000	test: 0.823

In [52]:
from sklearn.linear_model import RidgeClassifier

def stack_model(train_stack, test_stack, y):  #oof_set =[oof_1, oof_2, oof_3, ..., oof_n], predictions_set =[predictions_1, predictions_2, predictions_3, ..., predictions_n],

    oof = np.zeros((train_stack.shape[0],))
    predictions = np.zeros((test_stack.shape[0],))
    scores = []

    for fold_, (trn_idx, val_idx) in enumerate(kfold.split(train_stack, y)):
        trn_data, trn_y = train_stack[trn_idx], y[trn_idx]
        val_data, val_y = train_stack[val_idx], y[val_idx]

        clf = RidgeClassifier(random_state=2099)
        clf.fit(trn_data, trn_y)

        oof[val_idx] = clf._predict_proba_lr(val_data)[:,1]
        predictions +=clf._predict_proba_lr(test_stack)[:,1] / kfold.n_splits

        score_single = roc_auc_score(val_y, oof[val_idx])
        scores.append(score_single)
    print('mean: ', np.mean(scores))

    return oof, predictions



In [53]:
pred_matrix = np.hstack([sub_preds_1[:,np.newaxis], sub_preds_2[:,np.newaxis], sub_preds_3[:,np.newaxis]])
oof_matrix = np.hstack([oof_preds_1[:,np.newaxis], oof_preds_2[:,np.newaxis], oof_preds_3[:,np.newaxis]])

oof_stack, predictions_stack = stack_model(oof_matrix, pred_matrix, target)

mean:  0.8398889571015561


In [56]:
result = pd.DataFrame({'id':test.index, 'target': predictions_stack})
# predictions_stack.to_csv('./stack_model.csv', index)

In [57]:
result

,id,target
0,1000000,0.612764
1,1000001,0.359968
2,1000002,0.671185
3,1000003,0.660531
4,1000004,0.349935
...,...,...
995,1000995,0.494666
996,1000996,0.693508
997,1000997,0.451942
998,1000998,0.667875
